<a href="https://colab.research.google.com/github/Vivek2222/AI-Launchpad/blob/main/Day_3_Assignment_1_QA_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "langchain" "pypdf<=3.17.0" "faiss-cpu<=1.7.4" "openai>1" "tiktoken<=0.5.1"
!pip install streamlit
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
     

In [ ]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.5.1
    Uninstalling tiktoken-0.5.1:
      Successfully uninstalled tiktoken-0.5.1


In [ ]:
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0


In [7]:
#### App Implementation
%%writefile app.py

import streamlit as st
import os
import openai
import pandas as pd
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.chat_models import ChatOpenAI
#from langchain.llms import ChatOpenAI
#from langchain_openai import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
import tempfile  # To handle file saving for uploaded PDFs

# Streamlit UI elements for User Profile and Links
st.sidebar.markdown("## [Vivek Patil](https://www.linkedin.com/in/vivek-patil-vp/)")  # Replace with your LinkedIn profile URL
#st.sidebar.markdown("<div class="badge-base LI-profile-badge" data-locale="en_US" data-size="large" data-theme="dark" data-type="HORIZONTAL" data-vanity="vivek-patil-vp"
#data-version="v1"><a class="badge-base__link LI-simple-link" href="https://in.linkedin.com/in/vivek-patil-vp?trk=profile-badge">Vivek Patil</a></div>)

st.sidebar.markdown("Tester")
st.sidebar.markdown("""
A passionate tester and problem-solver, driven to ensure software excellence and seamless user experiences. Enthusiastic about AI learning and exploring emerging technologies.
""")

# Title for the app
st.title("QA Assistant for The Project")

# OpenAI API key input
OPENAI_API_KEY = st.text_input("Enter your OpenAI API Key:", type="password")
if OPENAI_API_KEY:
    openai.api_key = OPENAI_API_KEY
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# File uploader for PDF
uploaded_pdf = st.file_uploader("Upload the Requirements PDF", type="pdf")

# Display options
pd.set_option("display.max_colwidth", None)

#LLM_NAME = "gpt-3.5-turbo-instruct"
LLM_NAME = "gpt-4o-mini"

# Updated Prompt Template to reflect the new use case
PROMPT_TEMPLATE = """You are a QA Expert Assistant, specialized in analyzing, reviewing, and optimizing requirements for quality assurance.

Your role is to assist users by providing clear, actionable answers related to software quality assurance.
You will help enhance test plans, design test cases, improve coverage, and boost overall QA efficiency.
You have deep expertise in test automation frameworks and tools, including Selenium, Appium, Cypress, Robot Framework, Playwright, and TestComplete.
You are proficient in scripting languages like Python, Java, and JavaScript, and understand how to integrate automated testing into CI/CD pipelines.
You also possess knowledge across various domains, including healthcare, finance, and e-commerce, and are familiar with best practices and standards such as ISTQB and ISO guidelines.
You are expert in resource management, also in potential risks and mitigation strategies

When given a question and relevant excerpts from a Requirements Document, provide answers that include:
- Advanced QA strategies
- Potential risks and mitigation
- Automation recommendations
- Relevant coding examples

Additionally, suggest follow-up questions that a QA professional might ask to clarify requirements or ensure comprehensive testing.

Context:
{context}

Question:
{question}

Your detailed answer with QA insights, automation strategies, and coding recommendations:


"""

# Function to generate context storage from PDF chunks
def get_context_storage(pdf_file_path) -> FAISS:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, add_start_index=True)
    docs = PyPDFLoader(pdf_file_path).load_and_split(text_splitter)
    db = FAISS.from_documents(docs, OpenAIEmbeddings())
    return db

# Main application logic
if uploaded_pdf:
    st.write("Processing the uploaded PDF...")

    # Save the uploaded file temporarily
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_pdf_file:
        temp_pdf_file.write(uploaded_pdf.read())  # Save the uploaded file to disk
        temp_pdf_path = temp_pdf_file.name  # Get the path of the temporary file

    # Create the chain

    llm = ChatOpenAI(model=LLM_NAME)  # Use ChatOpenAI for chat models
    prompt = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["question", "context"])
    qa_chain = RetrievalQA.from_llm(llm=llm, retriever=get_context_storage(temp_pdf_path).as_retriever(), prompt=prompt)

    # Input for user's question
    question = st.text_input("Ask a question about the requirements:")

    # Display answer
    if question:
        answer = qa_chain(question)
        st.markdown("### Answer")
        st.markdown(f"**{answer}**")

    # Generate Test Plan Strategy
    if st.button("Generate Test Plan Strategy"):
        strategy_prompt = f"Create a comprehensive test plan strategy for the requirements provided in the document."
        strategy_response = qa_chain(strategy_prompt)
        st.markdown("### Test Plan Strategy")
        st.write(strategy_response)

    # Generate Test Data Requirements
    if st.button("Generate Test Data Requirements"):
        data_prompt = f"Analyze the requirements and generate test data requirements for the project."
        data_response = qa_chain(data_prompt)
        st.markdown("### Test Data Requirements")
        st.write(data_response)

    # Analyze Historical Test Cases
    historical_cases = st.file_uploader("Upload Historical Test Cases (CSV/Excel)", type=["csv", "xlsx"])
    if historical_cases:
        if st.button("Analyze Historical Test Cases"):
            # Read historical cases file
            historical_data = pd.read_csv(historical_cases) if historical_cases.name.endswith('.csv') else pd.read_excel(historical_cases)
            st.dataframe(historical_data)
            improvement_prompt = "Analyze the following historical test cases and suggest improvements:\n" + historical_data.to_string()
            improvement_response = qa_chain(improvement_prompt)
            st.markdown("### Suggested Improvements from Historical Releases")
            st.write(improvement_response)

    # Interactive Test Strategy Development
    st.markdown("### Build Your Test Strategy")
    iterative_question = st.text_input("Discuss your testing needs:")
    if iterative_question:
        iterative_response = qa_chain(iterative_question)
        st.markdown("**LLM Response**")
        st.write(iterative_response)

# Footer for App
st.markdown("""
---
*This QA Assistant is built by [Toni Ramchandani](https://www.linkedin.com/in/toni-ramchandani/).* Connect with me for more insights.
""")

Overwriting app.py


In [8]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 4s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [9]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.73.29.101

In [ ]:
# Start the Streamlit app
!streamlit run app.py &>/content/logs.txt &

# Use localtunnel to expose the app
!lt --port 8501

your url is: https://bitter-bushes-boil.loca.lt
